In [64]:
from elasticsearch import Elasticsearch, helpers
import csv
import json 
# Connexion au cluster Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")

# Nom de l'index
index_name = "review3"

# Mapping pour l'index 
mapping = {
    "mappings": {
        "_meta": {
            "created_by": "yaya"
        },
        "properties": {
            "@timestamp": {"type": "date"},
            "Company": {"type": "keyword"},
            "Customer": {"type": "keyword"},
            "Date_experience": {"type": "date", "format": "yyyy-MM-dd HH:mm:ss"},
            "Date_reply": {"type": "date", "format": "yyyy-MM-dd HH:mm:ss"},
            "Date_review": {"type": "date", "format": "yyyy-MM-dd HH:mm:ss"},
            "Experience": {"type": "keyword"},
            "Language": {"type": "keyword"},
            "Number_review": {"type": "long"},
            "Rating": {"type": "long"},
            "Reply": {"type": "keyword"},
            "Status": {"type": "keyword"},
            "Title": {"type": "keyword"},
            "column1": {"type": "long"},
            "document_id": {"type": "keyword"}  # Ajout d'un champ pour l'ID du document
        }
    }
}

# Créer l'index avec le mapping (you can skip this if the index already exists)
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)

# Lecture du fichier CSV et correction des champs vides
with open("../data/avis_clients.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    documents = []
    for idx, row in enumerate(reader):
        if "" in row:
            row["document_id"] = str(idx)  # Ajout de l'ID du document
            for key in list(row.keys()):
                if key == "":
                    row.pop(key)  # Suppression des champs avec des clés vides
        documents.append(row)

# Insérer les documents en vrac dans Elasticsearch
if documents:
    try:
        response = helpers.bulk(es, documents, index=index_name)
        print("Documents insérés avec succès :", response)
    except helpers.BulkIndexError as e:
        print("Erreur lors de l'insertion des documents :")
        for err in e.errors:
            print("Erreur :", err)

# Vérification des documents insérés
result = es.search(index=index_name, size=5)  # Récupère les 5 premiers documents

# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])


Documents insérés avec succès : (2394, [])
{'Company': 'Younited Credit', 'Customer': 'M françois GUYOT', 'Number_review': '2', 'Language': 'FR', 'Title': 'SUPER SOCIETE DE CREDITS TRES REACTIVE…', 'Date_review': '2023-07-28 06:28:34', 'Reply': 'No Reply', 'Date_reply': '2023-07-28 06:28:34', 'Rating': '5', 'Status': 'Verified', 'Experience': 'SUPER SOCIETE DE CREDITS TRES REACTIVE…SUPER SOCIETE DE CREDITS TRES REACTIVE sans problêmes merci pour votre pret accorde avec le moins d administratif possible', 'Date_experience': '2023-07-20 00:00:00', 'document_id': '0'}
{'Company': 'Younited Credit', 'Customer': 'Les parisiens', 'Number_review': '2', 'Language': 'FR', 'Title': 'Simple et efficace !', 'Date_review': '2023-07-28 17:28:34', 'Reply': 'No Reply', 'Date_reply': '2023-07-28 17:28:34', 'Rating': '5', 'Status': 'Verified', 'Experience': 'Simple et efficace !', 'Date_experience': '2023-07-19 00:00:00', 'document_id': '1'}
{'Company': 'Younited Credit', 'Customer': 'STEPHANIE RENOUX',

In [65]:
response = es.search(index="review3")
# Récupération du template
template = es.indices.get_mapping()

In [66]:
# recherche de tous les documents 
query = {
    "query": {
        "match_all": {}
    }
}

# Exécuter la requête
result = es.search(index="review3", body=query)

# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])


{'Company': 'Younited Credit', 'Customer': 'M françois GUYOT', 'Number_review': '2', 'Language': 'FR', 'Title': 'SUPER SOCIETE DE CREDITS TRES REACTIVE…', 'Date_review': '2023-07-28 06:28:34', 'Reply': 'No Reply', 'Date_reply': '2023-07-28 06:28:34', 'Rating': '5', 'Status': 'Verified', 'Experience': 'SUPER SOCIETE DE CREDITS TRES REACTIVE…SUPER SOCIETE DE CREDITS TRES REACTIVE sans problêmes merci pour votre pret accorde avec le moins d administratif possible', 'Date_experience': '2023-07-20 00:00:00', 'document_id': '0'}
{'Company': 'Younited Credit', 'Customer': 'Les parisiens', 'Number_review': '2', 'Language': 'FR', 'Title': 'Simple et efficace !', 'Date_review': '2023-07-28 17:28:34', 'Reply': 'No Reply', 'Date_reply': '2023-07-28 17:28:34', 'Rating': '5', 'Status': 'Verified', 'Experience': 'Simple et efficace !', 'Date_experience': '2023-07-19 00:00:00', 'document_id': '1'}
{'Company': 'Younited Credit', 'Customer': 'STEPHANIE RENOUX', 'Number_review': '3', 'Language': 'FR', 'T

C:\Users\yaya-\AppData\Local\Temp/ipykernel_25100/2743372402.py:9: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="review3", body=query)


In [67]:
# Requête pour obtenir les 5 mots clés positifs
query = {
    "query": {
        "match_all": {}
    },
    "aggs": {
        "positive_words": {
            "terms": {
                "field": "Experience",
                "size": 5,
                "min_doc_count": 5
            },
            "aggs": {
                "rating_filter": {
                    "filter": {
                        "range": {
                            "Rating": {
                                "gte": 5,
                                "lte": 5
                            }
                        }
                    }
                }
            }
        }
    }
}

# Exécution de la requête


# Exécution de la requête
result = es.search(index=index_name, body = query)

# Récupération des résultats
positive_words = result["aggregations"]["positive_words"]["buckets"]

for positive_word in positive_words:
    print(f"Mot clé positif : {positive_word['key']} - Nombre d'occurrences : {positive_word['doc_count']}")


Mot clé positif : Simple et rapide - Nombre d'occurrences : 52
Mot clé positif : Simple et efficace - Nombre d'occurrences : 31
Mot clé positif : Excellent service - Nombre d'occurrences : 27
Mot clé positif : Très bon service - Nombre d'occurrences : 26
Mot clé positif : Rapide et simple - Nombre d'occurrences : 20


C:\Users\yaya-\AppData\Local\Temp/ipykernel_25100/1432911214.py:33: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body = query)


In [68]:
query = {
    "query": {
        "match_all": {}
    },
    "aggs": {
        "negative_words": {
            "terms": {
                "field": "Title",
                "size": 10,
                "min_doc_count": 5
            },
            "aggs": {
                "rating_filter": {
                    "filter": {
                        "range": {
                            "Rating": {
                                "gte": 1,
                                "lte": 1
                            }
                        }
                    }
                }
            }
        }
    }
}
# Exécution de la requête
result = es.search(index=index_name, body=query)

# Récupération des résultats
negative_words = result["aggregations"]["negative_words"]["buckets"]

# Affichage des résultats
for negative_word in negative_words:
    print(f"Mot clé négatif : {negative_word['key']} - Nombre d'occurrences : {negative_word['doc_count']}")

Mot clé négatif : Simple et rapide - Nombre d'occurrences : 57
Mot clé négatif : Simple - Nombre d'occurrences : 48
Mot clé négatif : Simple et efficace - Nombre d'occurrences : 39
Mot clé négatif : Excellent - Nombre d'occurrences : 37
Mot clé négatif : Rapide - Nombre d'occurrences : 35
Mot clé négatif : Super - Nombre d'occurrences : 33
Mot clé négatif : Excellent service - Nombre d'occurrences : 29
Mot clé négatif : Très bon service - Nombre d'occurrences : 27
Mot clé négatif : Parfait - Nombre d'occurrences : 25
Mot clé négatif : Rapide et simple - Nombre d'occurrences : 24


C:\Users\yaya-\AppData\Local\Temp/ipykernel_25100/3111089838.py:28: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


# Importer Elasticsearch-py pour la gestion de l'API Elasticsearch
from elasticsearch import Elasticsearch

# Se connecter à Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")

# Définir un normalizer de minuscules
lowercase_normalizer = {
    "settings": {
        "analysis": {
            "normalizer": {
                "lowercase_normalizer": {
                    "type": "custom",
                    "filter": ["lowercase"]
                }
            }
        }
    }
}

# Appliquer le normalizer de minuscules à un champ
mapping = {
    "mappings": {
        "properties": {
            "Experience": {
                "type": "text",
                "normalizer": "lowercase_normalizer"
            }
        }
    }
}

# Créer l'index avec le mapping et le normalizer
es.indices.create(index="index_name", body={"settings": lowercase_normalizer})
es.indices.put_mapping(index="index_name", body=mapping)


query = {
    "query": {
        "match_all": {}
    },
    "aggs": {
        "mots_negatifs": {
            "terms": {
                "field": "Experience",
                "size": 10000,
                "include": [
                    "mauvais",
                    "terrible",
                    "déçu",
                    "pauvre",
                    "insatisfait",
                    "bad",
                    "avoid",
                    "arnaque",
                    "disaster",
                    "worse",
                    "incompétent"
                ],
                "analyzer": "lowercase_analyzer"
            }
        }
    }
}

result = es.search(index=index_name, body=query)

# Récupération des résultats
mots_negatifs = result["aggregations"]["mots_negatifs"]["buckets"]

# Affichage des résultats
for mot_negatif in mots_negatifs:
    print(f"Mot négatif : {mot_negatif['key']} - Nombre d'occurrences : {mot_negatif['doc_count']}")


In [69]:
# Requête d'agrégation pour obtenir des statistiques sur le champ "Rating"
query = {
    "size": 0,
    "aggs": {
        "value_count": {
            "value_count": {
                "field": "Rating"
            }
        },
        "rating_stats": {
            "stats": {
                "field": "Rating"
            }
        },
        "extended_ratings_stats": {
            "extended_stats": {
                "field": "Rating"
            }
        }
    }
}

# Exécuter la requête
result = es.search(index="review3", body=query)

# Afficher les résultats de l'agrégation
aggregations = result.get("aggregations", {})
value_count = aggregations.get("value_count", {})
rating_stats = aggregations.get("rating_stats", {})
extended_ratings_stats = aggregations.get("extended_ratings_stats", {})

print("Nombre total de valeurs de rating:", value_count.get("value", 0))
print("Statistiques de rating:")
print("    Minimum:", rating_stats.get("min", 0))
print("    Maximum:", rating_stats.get("max", 0))
print("    Moyenne:", rating_stats.get("avg", 0))
print("    Somme:", rating_stats.get("sum", 0))
print("    Écart-type:", extended_ratings_stats.get("std_deviation", 0))

Nombre total de valeurs de rating: 2394
Statistiques de rating:
    Minimum: 1.0
    Maximum: 5.0
    Moyenne: 4.7180451127819545
    Somme: 11295.0
    Écart-type: 0.7912112398310651


C:\Users\yaya-\AppData\Local\Temp/ipykernel_25100/2272750476.py:24: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="review3", body=query)


In [70]:
# Liste des entreprises avec le plus de commentaires négatifs
query = {
    "query": {
        "match_all": {}
    },
    "sort": [
        {
            "Rating": {
                "order": "asc"  # Tri ascendant (du plus bas au plus haut)
            }
        }
    ]
}

# Exécution de la requête pour récupérer les commentaires triés par note
result = es.search(index=index_name, body=query)

# Analyse des commentaires triés pour identifier les entreprises avec le plus de commentaires négatifs
from collections import Counter

negative_reviews = []
for hit in result["hits"]["hits"]:
    rating = int(hit["_source"]["Rating"])  # Convertir la note en entier
    if rating < 3:  # Supposons que 1 à 2 sont considérés comme des notes négatives
        negative_reviews.append(hit["_source"]["Company"])

# Utilisation de Counter pour compter le nombre de commentaires négatifs par entreprise
company_count = Counter(negative_reviews)

# Triez les entreprises par le nombre de commentaires négatifs (du plus au moins)
sorted_companies = sorted(company_count.items(), key=lambda x: x[1], reverse=True)

# Afficher les entreprises avec le plus de commentaires négatifs
for company, count in sorted_companies[:6]:  # Affiche les 5 premières entreprises avec le plus de commentaires négatifs
    print(f"Entreprise : {company}, Nombre de commentaires négatifs : {count}")


Entreprise : Younited Credit, Nombre de commentaires négatifs : 6
Entreprise : Orange Bank, Nombre de commentaires négatifs : 3
Entreprise : Cofidis, Nombre de commentaires négatifs : 1


C:\Users\yaya-\AppData\Local\Temp/ipykernel_25100/974567915.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


In [71]:
es = Elasticsearch(hosts="http://localhost:9200")

# Index des avis clients
index_name = "review3"

# Exécution de la requête
result = es.search(index=index_name, 
body={
    "query": {
        "match": {
            "Reply": {
                "query": "No Reply"
            }
        }
    },
    "aggs": {
        "companies": {
            "terms": {
                "field": "Company",
                "size": 10
            }
        }
    }
})

# Récupération des résultats
companies = result["aggregations"]["companies"]["buckets"]

# Affichage des résultats
for company in companies:
    print(f"Entreprise : {company['key']} - Nombre de No Reply : {company['doc_count']}")

C:\Users\yaya-\AppData\Local\Temp/ipykernel_25100/2866378900.py:7: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name,


Entreprise : Younited Credit - Nombre de No Reply : 820
Entreprise : Boursorama Banque - Nombre de No Reply : 762
Entreprise : Orange Bank - Nombre de No Reply : 176
Entreprise : Floabank - Nombre de No Reply : 82
Entreprise : Anytime - Nombre de No Reply : 48
Entreprise : Cofidis - Nombre de No Reply : 2


In [72]:
# Exécution de la requête pour obtenir le classement des entreprises selon leur note moyenne
query = {
    "aggs": {
        "companies": {
            "terms": {
                "field": "Company"
            },
            "aggs": {
                "average_rating": {
                    "avg": {
                        "field": "Rating"
                    }
                }
            }
        }
    }
}

result = es.search(index=index_name, body=query)

# Récupération des résultats
companies = result["aggregations"]["companies"]["buckets"]

# Tri des entreprises par leur note moyenne
companies.sort(key=lambda company: company["average_rating"]["value"], reverse=False)

# Affichage des résultats
for company in companies:
    average_ratings = round(company['average_rating']['value'], 2)
    print(f"Entreprise : {company['key']} - Note moyenne : {average_ratings}")


Entreprise : Floabank - Note moyenne : 4.49
Entreprise : Orange Bank - Note moyenne : 4.62
Entreprise : Boursorama Banque - Note moyenne : 4.67
Entreprise : Anytime - Note moyenne : 4.73
Entreprise : Younited Credit - Note moyenne : 4.82
Entreprise : Cofidis - Note moyenne : 4.86


C:\Users\yaya-\AppData\Local\Temp/ipykernel_25100/4117333319.py:19: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


In [73]:
# Exécution de la requête pour obtenir le groupe selon leur statut
query = {
    "aggs": {
        "statuses": {
            "terms": {
                "field": "Status"
            }
        }
    }
}

result = es.search(index=index_name, body=query)

# Récupération des résultats
statuses = result["aggregations"]["statuses"]["buckets"]

# Affichage des résultats
for status in statuses:
    print(f"Status : {status['key']} - Nombre de commentateurs : {status['doc_count']}")


Status : Invited - Nombre de commentateurs : 3632
Status : Verified - Nombre de commentateurs : 1054
Status : Redirected - Nombre de commentateurs : 102


C:\Users\yaya-\AppData\Local\Temp/ipykernel_25100/3781268496.py:12: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


In [74]:
# Exécution de la requête
query = {
    "query": {
        "match": {
            "Reply": "No Reply"
        }
    },
    "aggs": {
        "total_no_reply": {
            "value_count": {
                "field": "Reply"
            }
        },
        "companies": {
            "terms": {
                "field": "Company"
            },
            "aggs": {
                "number_of_no_reply": {
                    "value_count": {
                        "field": "Reply"
                    }
                }
            }
        }
    }
}

result = es.search(index=index_name, body=query)
# Récupération des résultats
total_no_reply = result["aggregations"]["total_no_reply"]["value"]
companies = result["aggregations"]["companies"]["buckets"]

# Affichage des résultats
for company in companies:
    number_of_no_reply = company["number_of_no_reply"]["value"]
    percentage = (number_of_no_reply / total_no_reply) * 100
    print(f"Entreprise : {company['key']} - Nombre de No Reply : {number_of_no_reply} - Taux de No Reply : {percentage:.2f}%")


Entreprise : Younited Credit - Nombre de No Reply : 820 - Taux de No Reply : 43.39%
Entreprise : Boursorama Banque - Nombre de No Reply : 762 - Taux de No Reply : 40.32%
Entreprise : Orange Bank - Nombre de No Reply : 176 - Taux de No Reply : 9.31%
Entreprise : Floabank - Nombre de No Reply : 82 - Taux de No Reply : 4.34%
Entreprise : Anytime - Nombre de No Reply : 48 - Taux de No Reply : 2.54%
Entreprise : Cofidis - Nombre de No Reply : 2 - Taux de No Reply : 0.11%


C:\Users\yaya-\AppData\Local\Temp/ipykernel_25100/3237135795.py:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)
